In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Reasoning Engine in Vertex AI - orchestration
@forusone (shins777@gmail.com)

This colab explains how to deal with prompt template for better query and reasoning

References :  
*  https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines
*  https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/develop#prompt-template
*  https://python.langchain.com/v0.2/docs/concepts/#prompt-templates



## Init Vertex AI SDK

In [1]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    "google-cloud-firestore" \
    "langchain-google-firestore" \
    "langgraph" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00


In [2]:
import sys
from IPython.display import display, Markdown

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Configuration and initialization

In [3]:
# Static configuration
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://reasoning_7424"
MODEL = "gemini-1.5-flash"

# Firestore
DATABASE="chat-history"
COLLECTION="history"

### Initialize Vertex AI

In [4]:
import vertexai
from vertexai.preview import reasoning_engines

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

## Firestore setting for chat history

### Helper method to get session history

In [5]:

# refer to : https://python.langchain.com/v0.2/docs/integrations/memory/google_firestore/#custom-client

def get_session_history(session_id: str):
    """
    Retrieve chat history data with session id for each conversation from Firestore on GCP
    """

    from langchain_google_firestore import FirestoreChatMessageHistory
    from google.cloud import firestore

    client = firestore.Client(project=PROJECT_ID, database=DATABASE)

    return FirestoreChatMessageHistory(
        client=client,
        session_id=session_id,
        collection=COLLECTION,
        encode_message=False,
    )

def get_converstion(chat_history):
  """
  Get conversation from chat history
  """

  from langchain.schema import HumanMessage, AIMessage
  import langchain_core.messages

  for message in chat_history.messages:
      if type(message) is HumanMessage:
        print(f"Human : {message.content}")
      elif type(message) is AIMessage:
        print(f"AI : {message.content}")

def interactive_chat(agent, query, session_id):
  """
  Chat with agent with chat history.
  """
  respone = agent.query(input=query,
                        config={"configurable": {"session_id": session_id}},
  )
  return respone['output']


### Unit test of Firestore

In [6]:
# Tool define
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
  """
     지정된 날짜의 두 통화 간 환율을 검색합니다.
  """

  import requests

  response = requests.get(
      f"https://api.frankfurter.app/{currency_date}",
      params={"from": currency_from, "to": currency_to},
  )
  return response.json()

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents.format_scratchpad.tools import format_to_tool_messages

custom_prompt_template = {
    "user_input": lambda x: x["input"],
    "history": lambda x: x["history"],
    "agent_scratchpad": lambda x: format_to_tool_messages(x["intermediate_steps"]),
} | ChatPromptTemplate.from_messages([
    ("placeholder", "{history}"),
    ("user", "{user_input}"),
    ("placeholder", "{agent_scratchpad}"),
])

In [8]:
def langgraph_builder(*, model, **kwargs):
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser
    from langgraph.graph import END, MessageGraph

    output_parser = StrOutputParser()

    planner = ChatPromptTemplate.from_template(
        "Generate an argument about: {input}"
    ) | model | output_parser

    pros = ChatPromptTemplate.from_template(
        "List the positive aspects of {input}"
    ) | model | output_parser

    cons = ChatPromptTemplate.from_template(
        "List the negative aspects of {input}"
    ) | model | output_parser

    summary = ChatPromptTemplate.from_template(
        "Input:{input}\nGenerate a final response given the critique",
    ) | model | output_parser

    builder = MessageGraph()

    builder.add_node("planner", planner)
    builder.add_node("pros", pros)
    builder.add_node("cons", cons)
    builder.add_node("summary", summary)

    builder.add_edge("planner", "pros")
    builder.add_edge("planner", "cons")
    builder.add_edge("pros", "summary")
    builder.add_edge("cons", "summary")
    builder.add_edge("summary", END)
    builder.set_entry_point("planner")

    return builder.compile()

agent = reasoning_engines.LangchainAgent(
    model=MODEL,
    runnable_builder=langgraph_builder,
)


In [ ]:
from time import perf_counter

query = "생성형 AI의 문제점에 대해서 설명해주세요."

t1_start = perf_counter()
response = agent.query(input={"role": "user", "content": query})
t1_end = perf_counter()

display(Markdown(f"Elapse time : {t1_end-t1_start}"))

for re in response:
  display(Markdown((re['kwargs']['content'])))
  print('======')

Elapse time : 15.940361179999968

생성형 AI의 문제점에 대해서 설명해주세요.

## 생성형 AI의 문제점: 빛과 그림자

생성형 AI는 혁신적인 기술로, 콘텐츠 제작과 정보 접근 방식을 크게 바꾸고 있습니다. 하지만 이 혁신에는 빛과 그림자, 즉 장점과 함께 해결해야 할 문제점들이 존재합니다.

**1. 윤리적 문제:**

* **저작권 침해:** 생성형 AI는 기존 콘텐츠를 학습하여 새로운 콘텐츠를 생성합니다. 이 과정에서 저작권 침해 논란이 발생할 수 있으며, 원작자의 권리 보호 문제가 대두됩니다.
* **가짜 정보 확산:** 생성형 AI는 사실과 허구를 구분하지 못하고, 가짜 뉴스나 허위 정보를 만들어낼 수 있습니다. 이는 사회적 혼란과 불신을 야기할 수 있습니다.
* **편견 및 차별:** 생성형 AI는 학습 데이터의 편견을 반영하여 차별적인 결과를 만들어낼 수 있습니다. 이는 사회적 불평등을 심화시키고 특정 집단에 대한 차별을 강화할 수 있습니다.

**2. 기술적 문제:**

* **데이터 편향:** 생성형 AI는 학습 데이터에 의존하기 때문에 데이터 편향 문제를 피할 수 없습니다. 이는 특정 관점이나 시각에 치우친 결과를 만들어낼 수 있습니다.
* **보안 및 프라이버시 문제:** 생성형 AI는 개인 정보를 포함한 대량의 데이터를 학습하는데, 이는 개인 정보 유출 및 보안 위협으로 이어질 수 있습니다.
* **오류 및 예측 불가능성:** 생성형 AI는 예측 불가능한 오류를 발생시킬 수 있으며, 이는 사용자에게 피해를 줄 수 있습니다.

**3. 사회적 문제:**

* **일자리 감소:** 생성형 AI는 자동화를 통해 일자리 감소를 야기할 수 있으며, 이는 사회적 불안과 경제적 불평등을 심화시킬 수 있습니다.
* **창의성 저하:** 생성형 AI에 의존하는 경우, 인간의 창의성과 독창성이 저하될 수 있습니다. 이는 새로운 아이디어와 혁신의 탄생을 저해할 수 있습니다.
* **인간관계 단절:** 생성형 AI는 사람과 사람 사이의 소통과 관계를 단절시킬 수 있으며, 이는 사회적 고립과 외로움을 심화시킬 수 있습니다.

**결론:**

생성형 AI는 혁신적인 기술이지만, 윤리적, 기술적, 사회적 문제점을 해결해야 합니다. 이러한 문제점을 해결하기 위해서는 기술 개발과 함께 사회적 논의와 규제가 필요합니다. 생성형 AI의 긍정적인 측면을 활용하면서, 부정적인 측면을 최소화하기 위해 노력해야 합니다.

## 생성형 AI의 문제점: 빛과 그림자에 대한 균형 잡힌 분석

생성형 AI의 문제점에 대해 잘 정리된 답변입니다. 다만, 몇 가지 추가적인 분석과 균형 잡힌 시각을 더하면 더욱 완성도 높은 답변이 될 수 있습니다.

**1. 윤리적 문제:**

* **저작권 침해**: 긍정적인 측면에서, 생성형 AI는 새로운 창작물을 만들어내는 데 기여할 수 있습니다. 예술 분야에서 새로운 스타일을 시도하거나, 기존 작품을 재해석하는 데 활용될 수 있습니다. 문제는 이러한 창작물이 원작자의 권리를 침해하지 않도록 명확한 저작권 규정과 윤리적 가이드라인이 필요하다는 것입니다. 
* **가짜 정보 확산**: 생성형 AI는 가짜 정보를 만들어낼 수 있지만, 동시에 가짜 정보를 식별하고 검증하는 데에도 활용될 수 있습니다. 예를 들어, AI는 가짜 뉴스를 감지하고 진실을 밝히는 데 도움을 줄 수 있습니다. 중요한 것은 가짜 정보에 대한 교육과 비판적 사고 능력을 함양하는 것입니다.
* **편견 및 차별**: 생성형 AI는 학습 데이터의 편향을 반영할 수 있지만, 이러한 편견을 해결하기 위한 노력 또한 중요합니다. 데이터 셋의 다양성을 확보하고, 알고리즘에 대한 꾸준한 점검과 수정을 통해 편견을 최소화할 수 있습니다.

**2. 기술적 문제:**

* **데이터 편향**: 데이터 편향 문제는 생성형 AI 뿐만 아니라 모든 인공지능 기술에 공통적인 문제입니다. 데이터 수집 및 처리 과정에서 객관성을 유지하고 다양한 관점을 포함하는 노력이 필요합니다.
* **보안 및 프라이버시 문제**: 생성형 AI는 개인 정보를 포함한 대량의 데이터를 학습하는 과정에서 보안 및 프라이버시 문제를 야기할 수 있습니다. 개인 정보 보호를 위한 강력한 보안 시스템 구축과 데이터 암호화, 사용자 동의를 통한 데이터 활용 등의 해결책이 필요합니다.
* **오류 및 예측 불가능성**: 생성형 AI는 예측 불가능한 오류를 발생시킬 수 있으며, 이는 사용자에게 피해를 줄 수 있습니다. 오류 발생 가능성을 최소화하기 위한 알고리즘 개발과 지속적인 모니터링 및 검증이 중요합니다.

**3. 사회적 문제:**

* **일자리 감소**: 자동화로 인한 일자리 감소는 피할 수 없는 현실이지만, 새로운 일자리 창출과 교육 시스템 개선을 통해 사회적 충격을 완화할 수 있습니다. 생성형 AI를 활용한 새로운 산업 분야의 성장과 새로운 기술을 요구하는 직업의 증가를 기대할 수 있습니다.
* **창의성 저하**: 생성형 AI가 창의성 저하를 가져온다는 주장은 일면 타당하지만, AI는 인간의 창의성을 증폭시키는 도구로도 활용될 수 있습니다. AI가 제공하는 새로운 아이디어를 바탕으로 인간은 더욱 독창적인 결과물을 창출할 수 있습니다. 
* **인간관계 단절**: 생성형 AI는 인간관계를 단절시키는 것이 아니라, 새로운 형태의 소통 방식을 제공할 수 있습니다. AI를 통해 다양한 사람들과 연결되고, 새로운 공동체를 형성할 수 있는 가능성이 존재합니다.

**결론**: 생성형 AI는 혁신적인 기술이지만, 긍정적인 측면과 부정적인 측면 모두를 인지하고, 균형 잡힌 시각으로 바라보는 것이 중요합니다. 윤리적, 기술적, 사회적 문제점을 해결하기 위한 지속적인 노력과 사회적 합의가 필요합니다. 

**추가적인 분석**:

* **AI 개발 및 활용에 대한 사회적 책임**: 생성형 AI 개발 및 활용에 있어 윤리적 책임을 강조하고, 관련 규제 및 가이드라인 마련을 통해 문제점을 최소화해야 합니다.
* **AI 교육의 중요성**: 생성형 AI를 올바르게 이해하고 활용하기 위해서는 AI에 대한 교육이 중요합니다. 비판적 사고 능력을 함양하고, AI의 한계와 가능성을 균형 있게 이해해야 합니다.

생성형 AI는 혁신적인 기술이며, 우리 사회에 큰 영향을 미칠 것입니다. 긍정적인 측면을 극대화하고 부정적인 측면을 최소화하기 위해 지속적인 노력과 사회적 합의가 필요합니다.


생성형 AI의 문제점에 대한 답변에서 긍정적인 측면을 찾기는 어렵습니다. 답변은 생성형 AI의 윤리적, 기술적, 사회적 문제점을 중점적으로 다루고 있으며, 긍정적인 측면에 대한 언급은 "혁신적인 기술"이라는 표현 외에는 찾아볼 수 없습니다. 

하지만 생성형 AI는 분명 여러 긍정적인 측면을 가지고 있습니다. 몇 가지 예시를 들어보겠습니다:

**1. 콘텐츠 제작의 효율성 증대:** 
* 생성형 AI는 다양한 콘텐츠 (글, 이미지, 음악 등)를 빠르고 효율적으로 제작하는 데 도움을 줄 수 있습니다.
* 예술가, 작가, 디자이너 등 창작 활동을 하는 사람들에게 새로운 영감을 제공하고, 작업 시간을 단축시킬 수 있습니다.

**2. 정보 접근성 향상:** 
* 생성형 AI는 복잡한 정보를 쉽게 이해할 수 있는 형태로 변환하거나 요약하여 정보 접근성을 높일 수 있습니다.
* 다양한 언어로 번역하거나, 이미지를 텍스트로 변환하는 등 정보 장벽을 낮추는 데 기여할 수 있습니다.

**3. 개인 맞춤형 서비스 제공:** 
* 생성형 AI는 사용자의 선호도와 필요에 맞춘 개인 맞춤형 서비스를 제공할 수 있습니다.
* 예를 들어, 개인의 취향에 맞는 음악 추천, 맞춤형 교육 콘텐츠 제공, 개인화된 쇼핑 제안 등이 가능합니다.

**4. 새로운 분야의 혁신:** 
* 생성형 AI는 의료, 과학, 교육 등 다양한 분야에서 혁신을 이끌어낼 수 있습니다.
* 새로운 약물 개발, 질병 진단, 교육 콘텐츠 개발 등에 활용될 수 있습니다.

생성형 AI는 긍정적이고 부정적인 측면을 모두 가지고 있는 기술입니다. 위에서 언급한 긍정적인 측면을 최대한 활용하면서, 윤리적 문제, 기술적 문제, 사회적 문제를 해결하기 위한 노력이 필요합니다. 

생성형 AI의 발전과 활용에 대한 지속적인 논의와 연구를 통해, 인류에게 더 큰 도움을 줄 수 있는 방향으로 발전시켜 나가야 할 것입니다. 


## 생성형 AI의 문제점: 빛과 그림자, 그리고 그 사이

생성형 AI는 콘텐츠 제작 방식을 혁신하고 정보 접근성을 향상시키는 등 여러 긍정적인 측면을 가지고 있습니다. 하지만 동시에 윤리적, 기술적, 사회적 문제점 또한 존재하며, 이러한 문제점들은 간과해서는 안 될 중요한 사안입니다. 

**1. 윤리적 문제:**

* **저작권 침해**: 생성형 AI는 기존 콘텐츠를 학습하여 새로운 콘텐츠를 생성하는 과정에서 저작권 침해 논란이 발생할 수 있습니다. 하지만 생성형 AI는 새로운 창작물을 만들어내는 데 기여할 수 있으며, 예술 분야에서 새로운 스타일을 시도하거나, 기존 작품을 재해석하는 데 활용될 수 있습니다. 중요한 것은 이러한 창작물이 원작자의 권리를 침해하지 않도록 명확한 저작권 규정과 윤리적 가이드라인을 마련하는 것입니다. 
* **가짜 정보 확산**: 생성형 AI는 사실과 허구를 구분하지 못하고, 가짜 뉴스나 허위 정보를 만들어낼 수 있습니다. 하지만 동시에 AI는 가짜 정보를 식별하고 검증하는 데에도 활용될 수 있습니다. 예를 들어, AI는 가짜 뉴스를 감지하고 진실을 밝히는 데 도움을 줄 수 있습니다. 가짜 정보에 대한 교육과 비판적 사고 능력을 함양하는 것이 중요합니다.
* **편견 및 차별**: 생성형 AI는 학습 데이터의 편견을 반영하여 차별적인 결과를 만들어낼 수 있습니다. 하지만 이러한 편견을 해결하기 위한 노력 또한 중요합니다. 데이터 셋의 다양성을 확보하고, 알고리즘에 대한 꾸준한 점검과 수정을 통해 편견을 최소화할 수 있습니다.

**2. 기술적 문제:**

* **데이터 편향**: 생성형 AI는 학습 데이터에 의존하기 때문에 데이터 편향 문제를 피할 수 없습니다. 이는 모든 인공지능 기술에 공통적인 문제이며, 데이터 수집 및 처리 과정에서 객관성을 유지하고 다양한 관점을 포함하는 노력이 필요합니다.
* **보안 및 프라이버시 문제**: 생성형 AI는 개인 정보를 포함한 대량의 데이터를 학습하는데, 이는 개인 정보 유출 및 보안 위협으로 이어질 수 있습니다. 개인 정보 보호를 위한 강력한 보안 시스템 구축과 데이터 암호화, 사용자 동의를 통한 데이터 활용 등의 해결책이 필요합니다.
* **오류 및 예측 불가능성**: 생성형 AI는 예측 불가능한 오류를 발생시킬 수 있으며, 이는 사용자에게 피해를 줄 수 있습니다. 오류 발생 가능성을 최소화하기 위한 알고리즘 개발과 지속적인 모니터링 및 검증이 중요합니다.

**3. 사회적 문제:**

* **일자리 감소**: 자동화로 인한 일자리 감소는 피할 수 없는 현실이지만, 새로운 일자리 창출과 교육 시스템 개선을 통해 사회적 충격을 완화할 수 있습니다. 생성형 AI를 활용한 새로운 산업 분야의 성장과 새로운 기술을 요구하는 직업의 증가를 기대할 수 있습니다.
* **창의성 저하**: 생성형 AI가 창의성 저하를 가져온다는 주장은 일면 타당하지만, AI는 인간의 창의성을 증폭시키는 도구로도 활용될 수 있습니다. AI가 제공하는 새로운 아이디어를 바탕으로 인간은 더욱 독창적인 결과물을 창출할 수 있습니다. 
* **인간관계 단절**: 생성형 AI는 인간관계를 단절시키는 것이 아니라, 새로운 형태의 소통 방식을 제공할 수 있습니다. AI를 통해 다양한 사람들과 연결되고, 새로운 공동체를 형성할 수 있는 가능성이 존재합니다.

**긍정적인 측면:**

* **콘텐츠 제작의 효율성 증대**: 생성형 AI는 다양한 콘텐츠 (글, 이미지, 음악 등)를 빠르고 효율적으로 제작하는 데 도움을 줄 수 있습니다. 예술가, 작가, 디자이너 등 창작 활동을 하는 사람들에게 새로운 영감을 제공하고, 작업 시간을 단축시킬 수 있습니다.
* **정보 접근성 향상**: 생성형 AI는 복잡한 정보를 쉽게 이해할 수 있는 형태로 변환하거나 요약하여 정보 접근성을 높일 수 있습니다. 다양한 언어로 번역하거나, 이미지를 텍스트로 변환하는 등 정보 장벽을 낮추는 데 기여할 수 있습니다.
* **개인 맞춤형 서비스 제공**: 생성형 AI는 사용자의 선호도와 필요에 맞춘 개인 맞춤형 서비스를 제공할 수 있습니다. 예를 들어, 개인의 취향에 맞는 음악 추천, 맞춤형 교육 콘텐츠 제공, 개인화된 쇼핑 제안 등이 가능합니다.
* **새로운 분야의 혁신**: 생성형 AI는 의료, 과학, 교육 등 다양한 분야에서 혁신을 이끌어낼 수 있습니다. 새로운 약물 개발, 질병 진단, 교육 콘텐츠 개발 등에 활용될 수 있습니다.

**결론:**

생성형 AI는 혁신적인 기술이지만, 긍정적인 측면과 부정적인 측면 모두를 인지하고, 균형 잡힌 시각으로 바라보는 것이 중요합니다. 윤리적, 기술적, 사회적 문제점을 해결하기 위한 지속적인 노력과 사회적 합의가 필요합니다. 

**추가적인 분석:**

* **AI 개발 및 활용에 대한 사회적 책임**: 생성형 AI 개발 및 활용에 있어 윤리적 책임을 강조하고, 관련 규제 및 가이드라인 마련을 통해 문제점을 최소화해야 합니다.
* **AI 교육의 중요성**: 생성형 AI를 올바르게 이해하고 활용하기 위해서는 AI에 대한 교육이 중요합니다. 비판적 사고 능력을 함양하고, AI의 한계와 가능성을 균형 있게 이해해야 합니다.

생성형 AI는 우리 사회에 큰 영향을 미칠 것입니다. 긍정적인 측면을 극대화하고 부정적인 측면을 최소화하기 위해 지속적인 노력과 사회적 합의가 필요합니다. 
